In [5]:
!pip install torchviz
!pip install graphviz

In [1]:
import torchviz
import torch
from deeppoly import DeepPoly, ReLuTransformer


In [2]:
paper_model = torch.nn.Sequential(
    torch.nn.Linear(2, 2),
    torch.nn.ReLU(),
    torch.nn.Linear(2, 2))
    #torch.nn.ReLU())
"""
    torch.nn.Linear(2, 2),
    torch.nn.Linear(2, 1)"""


#Init layers with paper weights and biases

#Linear layer 1
paper_model[0].weight.data = torch.tensor([[1.0, 1.0], [1.0, -1.0]])
paper_model[0].bias.data = torch.tensor([0.0, 0.0])

#Nothin  to init for RELU

#Linear layer 2
paper_model[2].weight.data = torch.tensor([[1.0, 1.0], [1.0, -1.0]])
paper_model[2].bias.data = torch.tensor([0.0, 0.0])

"""
#Nothing to init for RELU

#Linear layer 3
paper_model[4].weight.data = torch.tensor([[1.0, 1.0], [0.0, 1.0]])
paper_model[4].bias.data = torch.tensor([1.0, 0.0])


#Our network includes the verification layer
paper_model[5].weight.data = torch.tensor([[1.0, -1.0]])
paper_model[5].bias.data = torch.tensor([0.0])
"""

dp = DeepPoly(paper_model, input_size = 2)

lower_bounds_init = torch.tensor([-1.0, -1.0])
upper_bounds_init = torch.tensor([1.0, 1.0])


In [3]:
lr = 0.7
epochs = 10
step_size = 1
gamma = 0.75

for param in dp.verifier_net.parameters():
    param.requires_grad = False

for layer in dp.verifier_net:
    if isinstance(layer, ReLuTransformer):
        layer.beta.requires_grad = True
    

optimizer = torch.optim.Adam(
            dp.verifier_net.parameters(), 
            lr=lr # Learning rate
        )
    



In [4]:


### Epoch1 1
ver_lb, _ = dp(lower_bounds_init, upper_bounds_init)

for i in range(len(dp.verifier_net) - 2, -1, -1):
    print(f"Backsubstitute layer {i}")
    ver_lb, _ = dp.backsubstitute(i)


loss = -torch.sum(ver_lb)
print(f"Loss: {loss}")
torchviz.make_dot(loss, params=dict(dp.verifier_net.named_parameters()), show_attrs=True, show_saved=True).render('loss_graph_first_backwards_small', format='pdf')




optimizer.zero_grad()
loss.backward(retain_graph=True)
optimizer.step()

Backsubstitute layer 1
Backsubstitute layer 0
Loss: 3.0


ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [5]:
### Epoch1 2
ver_lb, _ = dp(lower_bounds_init, upper_bounds_init)

for i in range(len(dp.verifier_net) - 2, -1, -1):
    print(f"Backsubstitute layer {i}")
    ver_lb, _ = dp.backsubstitute(i)


loss = -torch.sum(ver_lb)
print(f"Loss: {loss}")
torchviz.make_dot(loss, params=dict(dp.verifier_net.named_parameters()), show_attrs=True, show_saved=True).render('loss_graph_second_backwards_small', format='pdf')



Backsubstitute layer 1
Backsubstitute layer 0
Loss: 3.0


In [6]:
for i in range(epochs):

    ### Epoch1 1
    ver_lb, _ = dp(lower_bounds_init, upper_bounds_init)

    for i in range(len(dp.verifier_net) - 2, -1, -1):
        print(f"Backsubstitute layer {i}")
        ver_lb, _ = dp.backsubstitute(i)


    loss = -torch.sum(ver_lb)
    print(f"Loss: {loss}")
    torchviz.make_dot(loss, params=dict(dp.verifier_net.named_parameters()), show_attrs=True, show_saved=True).render('loss_graph_first_backwards_small', format='pdf')




    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Backsubstitute layer 1
Backsubstitute layer 0
Loss: 3.161874771118164
Backsubstitute layer 1
Backsubstitute layer 0
Loss: 2.903167724609375
Backsubstitute layer 1
Backsubstitute layer 0
Loss: 2.622002124786377
Backsubstitute layer 1
Backsubstitute layer 0
Loss: 2.4573187828063965
Backsubstitute layer 1
Backsubstitute layer 0
Loss: 2.3393380641937256
Backsubstitute layer 1
Backsubstitute layer 0
Loss: 2.2389330863952637
Backsubstitute layer 1
Backsubstitute layer 0
Loss: 2.16337513923645
Backsubstitute layer 1
Backsubstitute layer 0
Loss: 2.110642910003662
Backsubstitute layer 1
Backsubstitute layer 0
Loss: 2.0753049850463867
Backsubstitute layer 1
Backsubstitute layer 0
Loss: 2.0519967079162598


In [2]:
beta_heuristic_l2_last = torch.tensor([1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
        1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0.,
        1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
        1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
        0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
        1., 0., 1., 1., 0., 1., 1., 0., 0., 0.])


beta_bounded_l2_last = torch.tensor([9.9941e-01, 9.9968e-01, 7.1386e-01, 5.9748e-04, 9.9954e-01, 9.9843e-01,
        9.9960e-01, 9.9937e-01, 9.9963e-01, 9.9953e-01, 9.9958e-01, 6.9436e-04,
        9.9948e-01, 9.9969e-01, 9.9964e-01, 9.9955e-01, 6.4342e-01, 7.7434e-04,
        9.9954e-01, 9.9959e-01, 9.9955e-01, 5.6680e-04, 9.9946e-01, 9.9956e-01,
        9.9959e-01, 9.9953e-01, 9.9965e-01, 9.9960e-01, 5.8461e-04, 9.9970e-01,
        9.9953e-01, 9.9959e-01, 9.9981e-01, 6.0869e-01, 9.7762e-01, 6.2044e-04,
        9.9952e-01, 7.0881e-01, 6.3539e-01, 6.2114e-01, 6.1086e-04, 6.2237e-04,
        7.6082e-04, 5.7363e-04, 9.9978e-01, 9.9941e-01, 6.4620e-04, 9.9956e-01,
        9.9849e-01, 6.5580e-04, 9.9950e-01, 2.5040e-03, 9.9957e-01, 6.5674e-01,
        9.9979e-01, 6.1877e-04, 5.7389e-01, 3.9351e-03, 9.9966e-01, 8.2860e-01,
        9.9947e-01, 6.7622e-04, 5.8735e-04, 6.6650e-04, 9.9954e-01, 9.9965e-01,
        7.2832e-01, 5.6642e-04, 7.0239e-04, 9.9950e-01, 9.7035e-01, 7.2578e-01,
        5.6024e-01, 9.9958e-01, 6.1510e-04, 5.6734e-04, 6.8079e-01, 9.9953e-01,
        9.9949e-01, 6.1387e-01, 9.9960e-01, 5.5061e-04, 5.8410e-04, 9.9966e-01,
        9.9943e-01, 9.9957e-01, 9.9936e-01, 9.9975e-01, 9.9958e-01, 5.2130e-04,
        3.4875e-03, 5.8689e-01, 9.9944e-01, 9.9960e-01, 6.3795e-04, 9.9955e-01,
        9.9948e-01, 5.2403e-01, 1.2484e-03, 6.4143e-04])


beta_gradient_l2_last = torch.tensor([-3.1719e-05, -5.0646e-04,  0.0000e+00,  2.0503e-03, -1.0593e-03,
        -4.3759e-05, -1.6328e-03, -3.9110e-04, -1.0940e-03, -1.2080e-02,
        -2.3141e-03,  4.0934e-04, -9.5891e-04, -4.1524e-04, -7.8657e-04,
        -2.9297e-03,  0.0000e+00,  7.2878e-04, -2.3891e-04, -5.3341e-04,
        -1.6257e-03,  8.4053e-03, -7.8474e-03, -1.5401e-04, -2.1737e-03,
        -2.6457e-04, -2.4015e-03, -1.4979e-04,  7.6027e-04, -1.8238e-04,
        -2.5395e-04, -6.8917e-04, -2.1457e-05,  0.0000e+00, -1.3561e-03,
         2.8951e-04, -1.9967e-03,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         3.4890e-03,  1.8449e-03,  1.4503e-03,  9.0523e-03, -9.6038e-05,
         0.0000e+00,  3.4449e-03, -3.0160e-03, -3.1366e-04,  3.6063e-03,
        -2.2454e-03,  1.2166e-04, -1.0047e-03,  0.0000e+00, -2.3420e-06,
         1.7678e-03,  0.0000e+00,  1.8851e-05, -1.6879e-04,  2.1244e-02,
        -1.3435e-03,  1.5103e-03,  2.5879e-04,  1.4018e-03, -9.7580e-04,
        -8.6202e-04,  0.0000e+00,  4.1891e-04,  1.2050e-04, -3.0163e-04,
         6.4264e-04,  0.0000e+00,  0.0000e+00, -2.6719e-04,  1.1868e-04,
         4.0239e-03,  0.0000e+00, -1.3216e-03, -1.2181e-02,  0.0000e+00,
        -2.4713e-03,  5.7272e-03,  2.7127e-04, -1.7047e-04, -7.7645e-04,
        -8.4178e-04, -1.5053e-03, -3.7765e-05, -5.4597e-05,  5.7422e-04,
         2.8289e-04,  0.0000e+00, -4.7336e-03, -1.9865e-03,  2.4007e-03,
        -1.4795e-03, -3.7343e-03,  0.0000e+00,  4.8381e-04,  1.5080e-03])

In [3]:
beta_bounded_l2_last - beta_heuristic_l2_last

tensor([-5.9003e-04, -3.2002e-04, -2.8614e-01,  5.9748e-04, -4.6003e-04,
        -1.5700e-03, -4.0001e-04, -6.3002e-04, -3.7003e-04, -4.6998e-04,
        -4.1997e-04,  6.9436e-04, -5.1999e-04, -3.1000e-04, -3.6001e-04,
        -4.5002e-04,  6.4342e-01,  7.7434e-04, -4.6003e-04, -4.1002e-04,
        -4.5002e-04,  5.6680e-04, -5.4002e-04, -4.4000e-04, -4.1002e-04,
        -4.6998e-04, -3.5000e-04, -4.0001e-04,  5.8461e-04, -2.9999e-04,
        -4.6998e-04, -4.1002e-04, -1.9002e-04,  6.0869e-01, -2.2380e-02,
         6.2044e-04, -4.8000e-04,  7.0881e-01, -3.6461e-01,  6.2114e-01,
         6.1086e-04,  6.2237e-04,  7.6082e-04,  5.7363e-04, -2.2000e-04,
         9.9941e-01,  6.4620e-04, -4.4000e-04, -1.5100e-03,  6.5580e-04,
        -5.0002e-04,  2.5040e-03, -4.2999e-04,  6.5674e-01, -2.0999e-04,
         6.1877e-04,  5.7389e-01,  3.9351e-03, -3.3998e-04,  8.2860e-01,
        -5.3000e-04,  6.7622e-04,  5.8735e-04,  6.6650e-04, -4.6003e-04,
        -3.5000e-04,  7.2832e-01,  5.6642e-04,  7.0